In [1]:
import build_network__
net = build_network__.BuildModel(1,1,5) # not statuc can be reset

Alpha Griffin TF_Curses Project


In [2]:
import tensorflow as tf
sess = tf.InteractiveSession()
import numpy as np
import os, sys, datetime

In [3]:
import ag.logging as log
log.set(5)

In [4]:
# you need to know your dataset sizes to make the placeholders
dataset_h = 66
dataset_w = 200
dataset_c = 3 
dataset_classes = 5

In [5]:
# get common stuff done
input_tensor, \
label_tensor, \
learn_rate, \
keep_prob = net.build_inputs(dataset_h, 
                             dataset_w, 
                             dataset_c, 
                             dataset_classes
                             )
#print("# Built Input {}".format(index))

In [6]:
# take the inputed variables and implement them
# input , convs, fc's, output_classes, prob
final_layer = net.build_outputs(input_tensor,
                                1,
                                1,
                                5,
                                keep_prob)

 ~D~ Start shape= Tensor("inputs/Placeholder:0", shape=(?, 66, 200, 3), dtype=float32)
 #I# Building 1 Convolutional Layers
 ~D~ Current Num of Features= 16
 ~D~ Start shape= Tensor("inputs/Placeholder:0", shape=(?, 66, 200, 3), dtype=float32)
 ~D~ Finished Building a conv Layer:
 ~ ~ 	Tensor("conv_layers/Relu:0", shape=(?, 66, 200, 16), dtype=float32)
 ~D~ Finishing Layer convLayer_0
 ~ ~ 	New Features = 3
 #I# Finished Layer convLayer_0
 # # 	New Features = 3
 #I# Finished Building 1 Conv Layers
 # # Moving To Flattening...
 #I# Tranitioning from Conv to fc layers... 16
 #I# Finished Flattening Layers
 #I# Building 1 Fully Connected Layers


In [7]:
train, loss, softmax = net.training_method(final_layer, label_tensor, learn_rate)

new test


In [ ]:
job_name = "AlphaGriffin_TF_Server"
known_machines = []
# this is another process... worker...
known_machines.append('localhost:2222')
# this is this process
known_machines.append('localhost:2223')
# known_machines.append( {'cool_name': ['sock_ip:port', 'localhost:2223']} )
known_machines_dict = { '{}'.format(job_name): [x for x in known_machines] }
len_machines = len(known_machines)
print("There are {} known machines".format(len_machines))
print(known_machines_dict)

In [ ]:
# cluster = tf.train.ClusterSpec(known_machines_dict)

In [ ]:
# this is the HOME position... "you are here"
#server = tf.train.Server(cluster,
#                         job_name,
#                         task_index=1) 
server = known_machines[0]
print(server)

In [ ]:
class data_prep(object):
    def __init__(self, images, labels):
        self.index_in_epoch = 0
        self.num_examples = 0 
        self.epochs_completed = 0
        self.num_examples = 0
        self.images = images
        self.labels = labels
        self.check_data()
        
    def check_data(self):
        try:
            assert self.images.shape[0] == self.labels.shape[0]
        except Exception as e:
            print("{} is not {}".format(self.images.shape[0], self.labels.shape[0]))
        self.num_examples = self.images.shape[0]
        return True
        

    def next_batch(self, batch_size, shuffle=False):
            """ Shuffle is off by default """
            start = self.index_in_epoch
            self.index_in_epoch += batch_size
            if self.index_in_epoch > self.num_examples:
                # Finished epoch                                                                                       
                self.epochs_completed += 1
                # Shuffle the data                                                                                     
                if shuffle:
                    perm = np.arange(self.num_examples) # should add some sort of seeding for verification            
                    np.random.shuffle(perm)
                    self.images = self.images[perm]
                    self.labels = self.labels[perm]
                # Start next epoch                                                                                     
                start = 0
                self.index_in_epoch = batch_size
                assert batch_size <= self.num_examples
            end = self.index_in_epoch
            return self.images[start:end], self.labels[start:end]

In [ ]:
load = np.load("/pub/dataset/mario/mariokart64_dataset_0.npz")
imgs = load['images']
labels = load['labels']
dataset_examples = imgs.shape[0]
dataset_h = imgs.shape[1]
dataset_w = imgs.shape[2]
dataset_c = imgs.shape[3]
dataset_classes = labels.shape[1]
dataset_shape = imgs[0].shape
label_example = labels[0]

In [ ]:
log.info("Images: {}".format(imgs.shape))
log.info("Labels: {}".format(labels.shape))
log.info("Image shape: {}".format(dataset_shape))
log.info("Label: {}".format(label_example))

In [ ]:
batcher = data_prep(imgs, labels)
print(batcher.images.shape[:])
test_batch = batcher.next_batch(50)
print(test_batch[0].shape[:])

In [ ]:
###### do a quick train and bail out for a solid save and reload...
iters = 5
batch_size = 100
# with tf.Session("grpc://{}".format(server)) as sess:
# sess = tf.InteractiveSession()
sess.run(tf.global_variables_initializer())
for training_step in range(iters):
    batch = batcher.next_batch(batch_size)
    feed_dict = {input_tensor: batch[0], label_tensor: batch[1], keep_prob: 0.8}
    sess.run(train, feed_dict=feed_dict)
    print("Completed Step: {}".format(training_step))
    # if training_step % 25-1 == 0:
    #    loss_value = loss.eval(feed_dict={input_tensor: batch[0], label_tensor: batch[1], keep_prob: 0.8})
    #    print("step: {} loss: {}".format(training_step, loss_value))

In [ ]:
# for each worker/server/w/e in the KNOWN CLUSTER... so job name would be known here.
#index = 0
"""
def build_graphs(num_conv, num_fc, num_outputs):
    all_graphs = []
    for index in range(len_machines):
        current_context = tf.Graph()
        current_worker = "/job:{}/task:{}".format(job_name, index)
        print("With {} build graph {}".format(current_worker, index))
        with tf.device(tf.train.replica_device_setter(worker_device=current_worker,
                                                      cluster=cluster)):
            # get common stuff done
            input_tensor, \
            label_tensor, \
            learn_rate, \
            keep_prob = net.build_inputs(dataset_h, 
                                         dataset_w, 
                                         dataset_c, 
                                         dataset_classes
                                         )
            print("# Built Input {}".format(index))
            
            # take the inputed variables and implement them
            x_image = net.build_outputs(input_tensor,
                                        num_conv,
                                        num_fc,
                                        num_outputs,
                                        keep_prob)
            
            print("# Built Output {}".format(index))
            
            # build training ops
            train_op, loss, softmax = net.training_method(x_image, 
                                                          input_label,
                                                          learn_rate
                                                          )
            # Finished Building Graph            
        all_graphs.append(current_context)
    return all_graphs
"""

In [ ]:
# mastergraph = build_graphs(5,5,5)
# print(mastergraph)